<a href="https://colab.research.google.com/github/albey-code/hippoabstraction/blob/main/GLM_FirstLevelModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook only contains the **no patch** analysis!

In [1]:
! pip install nilearn
import pandas as pd
import nibabel as nib
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 120.4 MB/s eta 0:00:00


In [6]:
# Mount Google Drive to access files

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
base_path = '/content/drive/MyDrive'

In [8]:
# Define subjects and runs

subjects = [f"subject_{i:02d}" for i in range(1, 24)]  #subject_01 to subject_23
runs = [1, 2, 3]

In [9]:
# Initialize dictionaries to hold paths for each subject

bold_paths = {}
confound_paths = {}
event_paths = {}

In [10]:
# Build paths automatically

for subj in subjects:
    subj_num = subj[-2:]  # Extract '01', '02', etc.

    # fMRI timeseries
    bold_paths[subj] = [
        f"{base_path}/neural_timeseries_fMRI/{subj}/sub{subj_num}_run{run}_bold.nii.gz"
        for run in runs
    ]

    # Physio/motion confound regressors
    confound_paths[subj] = [
        f"{base_path}/physio_motion_regressors/{subj}/sub-{subj_num}_run-{run}_motion_physio.tsv"
        for run in runs
    ]

    # Event files (NO-PATCH ONLY!)
    event_paths[subj] = [
        f"{base_path}/events/{subj}/sub{subj_num}_run{run}_events_np.tsv" #Here, the ending is different!
        for run in runs
    ]

In [11]:
# Sanity check: verify that all files exist

print("Checking file existence across all subjects and runs...\n")

for subj in subjects:
    for run_idx, (b, c, e) in enumerate(zip(bold_paths[subj], confound_paths[subj], event_paths[subj]), 1):
        missing = []
        for f in [b, c, e]:
            if not os.path.exists(f):
                missing.append(f)
        if missing:
            print(f"Missing files for {subj}, run {run_idx}:")
            for f in missing:
                print("   ", f)

print("Check complete.")

Checking file existence across all subjects and runs...

Check complete.


In [12]:
# Define output directory for betas

betas_dir = os.path.join(base_path, "betas_nopatch")
os.makedirs(betas_dir, exist_ok=True)

**General Linear Model (GLM)**

For more info. on the design decisions of the GLM, please see Denis' message on Mattermost, and Mona's reply via e-mail. E.g., it makes more sense to apply the noise model for the individual runs and to not apply signal scaling!

In [13]:
from nilearn.glm.first_level import FirstLevelModel

# GLM based on Garvert et al.'s (2017) Matlab script

first_level_model = FirstLevelModel(
    t_r=3.01,
    slice_time_ref=15/43,     # Equivalent to SPM’s fmri_t0 = 15 with 43 slices
    hrf_model='spm',
    high_pass=1/128,          # Nilearn uses Hz, so 1/128 s #See line 205 of mydesign_1012.m
    noise_model='ar1',        # See fmri_spec.cvi = 'AR(1)'
    standardize=False,        # No mention of mean 0 SD 1, so I disabled it
    signal_scaling=False,     # To match SPM’s default (no scaling)
    minimize_memory=False
)

# Fit the GLM

Here, I fit the GLM individually for runs 1, 2 and 3.

In [27]:
for subj in subjects:
    print(f"\nProcessing {subj}...")

    for run_idx, (bold_file, confounds_file, events_file) in enumerate(
        zip(bold_paths[subj], confound_paths[subj], event_paths[subj]), 1
    ):
        print(f"  Fitting run {run_idx}...")

        # Load events and confounds
        events = pd.read_csv(events_file, sep="\t")
        confounds = pd.read_csv(confounds_file, sep="\t", header=None) #Important! Set header=None, otherwise mismatch error!!

        # Fit GLM
        fmri_glm = first_level_model.fit(bold_file, events=events, confounds=confounds)

        # Design matrix
        design_matrix = fmri_glm.design_matrices_[0]

        # Ensure all column names are strings (fix for AttributeError)
        design_matrix.columns = design_matrix.columns.astype(str)

        design_dir = os.path.join(base_path, "design_matrices_nopatch", subj)
        os.makedirs(design_dir, exist_ok=True)
        design_matrix.to_csv(os.path.join(design_dir, f"design_run{run_idx}.csv"))

        # Create a run-specific folder for betas
        run_dir = os.path.join(betas_dir, subj, f"run{run_idx}_betas_np")
        os.makedirs(run_dir, exist_ok=True)

        # Save betas only for your task regressors #Otherwise it gets too cluttered in the Google Drive
        task_regressors = events['trial_type'].unique().tolist()
        print(f"Task regressors for {subj}, run {run_idx}: {task_regressors}")

        for regressor in design_matrix.columns:
            reg_str = str(regressor)

            # Skip all regressors not in the event file (i.e. confounds, drifts, constants)
            if reg_str not in task_regressors:
                continue

            beta_img = fmri_glm.compute_contrast(regressor, output_type="effect_size")
            beta_fname = f"sub-{subj[-2:]}_run{run_idx}_{regressor}_beta_np.nii.gz"
            beta_img.to_filename(os.path.join(run_dir, beta_fname))

print(f"  → Saved betas for {len(task_regressors)} regressors to {run_dir}")



Processing subject_01...
  Fitting run 1...
Task regressors for subject_01, run 1: ['button_press', 'object_6', 'object_4', 'object_2', 'object_7', 'object_8', 'object_10', 'object_9']
  Fitting run 2...
Task regressors for subject_01, run 2: ['object_8', 'object_10', 'button_press', 'object_7', 'object_9', 'object_4', 'object_6', 'object_2']
  Fitting run 3...
Task regressors for subject_01, run 3: ['object_9', 'object_2', 'object_4', 'button_press', 'object_6', 'object_7', 'object_8', 'object_10']

Processing subject_02...
  Fitting run 1...
Task regressors for subject_02, run 1: ['object_7', 'object_6', 'object_10', 'object_9', 'button_press', 'object_8', 'object_2', 'object_4']
  Fitting run 2...
Task regressors for subject_02, run 2: ['object_6', 'object_2', 'object_7', 'object_9', 'object_8', 'button_press', 'object_4', 'object_10']
  Fitting run 3...
Task regressors for subject_02, run 3: ['object_2', 'object_6', 'button_press', 'object_8', 'object_4', 'object_9', 'object_10', 

/tmp/ipython-input-3889021362.py:14: UserWarning: Matrix is singular at working precision, regularizing...
  fmri_glm = first_level_model.fit(bold_file, events=events, confounds=confounds)


Task regressors for subject_22, run 1: ['object_10', 'object_6', 'button_press', 'object_9', 'object_2', 'object_7', 'object_8', 'object_4']
  Fitting run 2...
Task regressors for subject_22, run 2: ['object_2', 'object_10', 'object_6', 'object_4', 'button_press', 'object_7', 'object_8', 'object_9']
  Fitting run 3...
Task regressors for subject_22, run 3: ['object_10', 'button_press', 'object_2', 'object_9', 'object_8', 'object_4', 'object_6', 'object_7']

Processing subject_23...
  Fitting run 1...
Task regressors for subject_23, run 1: ['object_10', 'object_8', 'object_9', 'object_4', 'button_press', 'object_2', 'object_7', 'object_6']
  Fitting run 2...
Task regressors for subject_23, run 2: ['object_8', 'button_press', 'object_6', 'object_10', 'object_9', 'object_2', 'object_4', 'object_7']
  Fitting run 3...
Task regressors for subject_23, run 3: ['object_7', 'object_8', 'object_6', 'object_9', 'object_2', 'object_10', 'object_4', 'button_press']
  → Saved betas for 8 regressors 

# Mean Run 1-3

In [4]:
from nilearn.image import mean_img
from glob import glob

In [3]:
base_path = '/content/drive/MyDrive'
betas_dir = os.path.join(base_path, "betas_nopatch")

In [16]:
for subj in subjects:
    print(f"\nAveraging betas for {subj}...")

    subj_dir = os.path.join(betas_dir, subj)
    mean_dir = os.path.join(subj_dir, "mean_betas_np")
    os.makedirs(mean_dir, exist_ok=True)

    # Find all run folders
    run_dirs = [os.path.join(subj_dir, d) for d in os.listdir(subj_dir) if d.startswith("run")]

    # Detect object labels automatically from run1 filenames
    run1_dir = run_dirs[0]
    object_files = [f for f in os.listdir(run1_dir) if f.endswith("_beta_np.nii.gz")]

    # Extract object label robustly (from filenames like 'sub-15_run1_object_2_beta_np.nii.gz')
    object_labels = []
    for f in object_files:
        # get the bit between 'run1_' and '_beta_np'
        try:
            label = f.split("_run")[1].split("_beta_np")[0]
            label = label.split("_", 1)[1]  # remove the run number (e.g., '1_')
            object_labels.append(label)
        except Exception as e:
            print(f"Could not parse label from {f}: {e}")

    object_labels = sorted(set(object_labels))
    print(f"  Found {len(object_labels)} object labels: {object_labels}")

    # --- Average betas across runs for each object ---
    for obj_label in object_labels:
        obj_imgs = []
        for run_dir in run_dirs:
            # Match any run for this object
            pattern = f"*_{obj_label}_beta_np.nii.gz"
            files = glob(os.path.join(run_dir, pattern))
            if files:
                obj_imgs.extend(files)

        if len(obj_imgs) == 0:
            print(f"No betas found for {obj_label} in {subj}")
            continue

        # Compute voxelwise mean
        mean_beta = mean_img(obj_imgs)

        # Save averaged beta map
        out_path = os.path.join(mean_dir, f"sub-{subj[-2:]}_{obj_label}_mean_beta_np.nii.gz")
        mean_beta.to_filename(out_path)

        print(f"  → Saved {obj_label} mean beta ({len(obj_imgs)} runs) to {out_path}")




Averaging betas for subject_01...
  Found 8 object labels: ['button_press', 'object_10', 'object_2', 'object_4', 'object_6', 'object_7', 'object_8', 'object_9']


/tmp/ipython-input-3690530966.py:44: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  mean_beta = mean_img(obj_imgs)


  → Saved button_press mean beta (3 runs) to /content/drive/MyDrive/betas_nopatch/subject_01/mean_betas_np/sub-01_button_press_mean_beta_np.nii.gz
  → Saved object_10 mean beta (3 runs) to /content/drive/MyDrive/betas_nopatch/subject_01/mean_betas_np/sub-01_object_10_mean_beta_np.nii.gz
  → Saved object_2 mean beta (3 runs) to /content/drive/MyDrive/betas_nopatch/subject_01/mean_betas_np/sub-01_object_2_mean_beta_np.nii.gz
  → Saved object_4 mean beta (3 runs) to /content/drive/MyDrive/betas_nopatch/subject_01/mean_betas_np/sub-01_object_4_mean_beta_np.nii.gz
  → Saved object_6 mean beta (3 runs) to /content/drive/MyDrive/betas_nopatch/subject_01/mean_betas_np/sub-01_object_6_mean_beta_np.nii.gz
  → Saved object_7 mean beta (3 runs) to /content/drive/MyDrive/betas_nopatch/subject_01/mean_betas_np/sub-01_object_7_mean_beta_np.nii.gz
  → Saved object_8 mean beta (3 runs) to /content/drive/MyDrive/betas_nopatch/subject_01/mean_betas_np/sub-01_object_8_mean_beta_np.nii.gz
  → Saved object_

# Concatenated Runs 1-3 (Optional)

This part is an alternative way to concatenate the raw neural timeseries for runs 1, 2 and 3. Then, specify a run regressor in the GLM, so that it knows which run is which as a constant of 1s for a certain run and 0s for the other two. But! this means that I will need to re-run the GLM with aforementioned run regressor. In the end, I would get a concatenated version of the betas (instead of a mean beta for each object and each voxel).